In [ ]:
import json
import math
import operator
from collections import OrderedDict
from glob import glob

import pandas as pd

## Generate value_maps.json

Before using this cell, please, save a copy of the ODS file as CSV with name `value_maps.csv`.

In [ ]:
df = pd.read_csv('value_maps.csv', header=0, skiprows=3)
for col in df.columns.values:
    if col.startswith('Unnamed'):
        del df[col]


def from_fractional(x):
    if '/' in x:
        num, den = x.split('/')
        return float(num) / float(den)
    else:
        return float(x)


def clean_EFXthC(x):
    if x == 'off':
        return float(999)
    else:
        return float(x)


value_maps_json = {}
for col in df.columns.values:
    if col == 'CLA000':
        d = (df[col] * 0.01).tolist()
    elif col == 'OSCprt':
        d = [from_fractional(x) for x in df[col]]
    elif col == 'EFXthC':
        d = [clean_EFXthC(x) for x in df[col]]
    else:
        d = [x for x in df[col]]
    # weird hack to leave only 3 decimals
    value_maps_json[col] = [float(f'{round(x, 3):.3f}') for x in d if not math.isnan(x)]
value_maps_json['MID000'] = [x for x in range(128)]
value_maps_json['BUT002'] = [x for x in range(2)]
value_maps_json['BUT003'] = [x for x in range(3)]
value_maps_json['BUT004'] = [x for x in range(4)]
value_maps_json['BUT005'] = [x for x in range(5)]
value_maps_json['BUT006'] = [x for x in range(6)]
value_maps_json['BUT007'] = [x for x in range(7)]
value_maps_json['BUT008'] = [x for x in range(8)]

value_maps_json = OrderedDict(sorted(value_maps_json.items(), key=operator.itemgetter(0)))
with open('../pch2csd/resources/value_maps.json', 'w') as f:
    json.dump(value_maps_json, f, indent=True)

## Convert old mappings to JSON

### mod_id.json

In [ ]:
mod_map = OrderedDict()
with open('ModID2Name.txt', 'r') as f:
    for line in f:
        t = line.split()
        if len(t) == 1:
            mod_map[t[0]] = 'Unknown'
        elif len(t) == 2:
            mod_map[t[0]] = t[1]
        else:
            raise RuntimeError(t)
with open('mod_id.json', 'w') as f:
    json.dump(mod_map, f)

### mod_id_connections.json

In [ ]:
mod_id_connections = OrderedDict()
for fn in glob('IO/*.txt'):
    id = int(fn.replace('IO/', '').replace('.txt', ''))
    data = {'inputs': [], 'outputs': []}
    with open(fn, 'r') as f:
        for line in f:
            if line == '':
                continue
            k, v = line.split()
            if k == 'I':
                data['inputs'].append(v)
            elif k == 'O':
                data['outputs'].append(v)
    mod_id_connections[id] = data
with open('mod_id_connections.json', 'w') as f:
    json.dump(mod_id_connections, f)